## Writing Viterbi Algorithm for the Primer

In [10]:
import numpy as np
import math

# Nucleotides and states
nucs = ['A', 'C', 'G', 'T']
states = ['E', '5', 'I']

# Initial probabilities
init_prob = {'E': 1.0, '5': 0.0, 'I': 0.0}

# Transition matrix
trans_prob = {
    'Start': {'E': 1.0, '5': 0.0, 'I': 0.0, 'End': 0.0},
    'E': {'E': 0.9, '5': 0.1, 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0, 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0, 'I': 0.9, 'End': 0.1}
}

# Emission probabilities
emit_prob = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.00, 'G': 0.95, 'T': 0.00},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},
}

# Logarithm function with protection for zero
def log(x):
    return -math.inf if x == 0 else math.log(x)

# Log probability calculation
def log_prob(path, seq):
    log_prob_value = 0.0
    if len(path) != len(seq):
        raise ValueError("The length of observed and state sequences must be the same")
    
    prev_state = 'Start'
    
    for i in range(len(seq)):
        curr_state = path[i]
        observed_nuc = seq[i]
        log_prob_value += log(trans_prob[prev_state][curr_state]) + log(emit_prob[curr_state][observed_nuc])
        prev_state = curr_state
        
    if prev_state == 'I':
        log_prob_value += log(trans_prob[prev_state]['End'])

    return log_prob_value

In [12]:
STATE_PATH = "EEEEEEEEEEEEEEEEEE5IIIIIII"
OBSERVED_SEQ = "CTTCATGTGAAAGCAGACGTAAGTCA"
ans = log_prob(STATE_PATH, OBSERVED_SEQ)

print(ans)

-41.21967768602254


In [14]:
def viterbi(obs_seq):
    num_states = len(states)
    num_obs = len(obs_seq)

    vmat = np.full((num_states, num_obs), -np.inf)
    backtrack = np.zeros((num_states, num_obs), dtype=int)

    state_index = {s: i for i, s in enumerate(states)}

    # Initialization using 'Start' transitions
    for i, state in enumerate(states):
        vmat[i, 0] = log(trans_prob['Start'][state]) + log(emit_prob[state][obs_seq[0]])

    # Recursion
    for t in range(1, num_obs):
        for j, curr_state in enumerate(states):
            max_val = -np.inf
            prev_best = 0
            for i, prev_state in enumerate(states):
                val = vmat[i, t - 1] + log(trans_prob[prev_state][curr_state])
                if val > max_val:
                    max_val = val
                    prev_best = i
            vmat[j, t] = max_val + log(emit_prob[curr_state][obs_seq[t]])
            backtrack[j, t] = prev_best

    # Termination
    last_probs = vmat[:, -1]
    if 'End' in trans_prob[states[0]]:
        for i, state in enumerate(states):
            if 'End' in trans_prob[state]:
                last_probs[i] += log(trans_prob[state]['End'])

    final_state = np.argmax(last_probs)
    best_path = [states[final_state]]

    # Backtracking
    for t in range(num_obs - 1, 0, -1):
        final_state = backtrack[final_state, t]
        best_path.insert(0, states[final_state])

    return best_path, np.max(last_probs)
obs_seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
path, prob = viterbi(obs_seq)

print("Most likely state path:", path)
print("Log probability:", prob)


Most likely state path: ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '5', 'I', 'I', 'I', 'I', 'I', 'I', 'I']
Log probability: -41.21967768602254
